In [1]:
### Imports from algo_common

import csv
import datetime
from dateutil import tz
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
import numpy as np
import os
import pandas as pd
import pytz
import sys
import time

os.chdir(r'D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy')
sys.path.append(os.path.join(os.getcwd(), '..'))

from trader_factory import build_active_IBridgePy_plus
from IBridgePy.IbridgepyTools import symbol

from Utils import Algos
from Utils import EnterExits
from Utils import IndicatorCalculations
from Utils import Orders
from Utils import Plotters
from Utils import Utils
from UnitTests import testutils








In [8]:
import datetime

test1 = datetime.datetime.now()

In [18]:
test2=datetime.datetime.now()

print(test2.timestamp()-test1.timestamp())
# print(type(test1))

156.3924160003662


In [16]:
test2.timestamp()

1670205667.928723

In [ ]:
sim = build_active_IBridgePy_plus('DU2628075') # Build an active trader to get connected to Interactive Brokers 

In [ ]:
min1Data=sim.request_historical_data(symbol('SPY'), '1 min', '20 D')
min5Data=sim.request_historical_data(symbol('SPY'), '5 mins', '20 D')
# min1Data.to_csv(r"D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Log\testData.csv")
# min5Data.to_csv(r"D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Log\testData5.csv")

In [ ]:
context = testutils.context(sim)
context.sTime = sim.get_datetime('US/Eastern') 

dailyPriceData = sim.request_historical_data(symbol('SPY'), '1 min', '23400 S')

dailyPriceData.to_csv(r"log\Daily_Price_Data_"+str(context.sTime.year)+"_"+str(context.sTime.month)+"_"+str(context.sTime.day)+".csv")

In [ ]:
min1Data=sim.request_historical_data(symbol('SPY'), '1 secs', '2000 S')
min5Data=sim.request_historical_data(symbol('SPY'), '5 secs', '2000 S')

nVec = [13, 48, 8]

maxPeriod=5*26*9 # max start periods for 5 minute data at 26 period filter (not quite sure about the 9 yet)
price, time, enter_flag, entry_flag, exit_flag, long_bool, short_bool, =[[]*i for i in range(7)]


# Initialize structure (use dataframe for now) # TODO make a class eventually
ema=pd.DataFrame(columns=["fast", "fast_n", "fast_1", "fast_2", 
                  "slow", "slow_n", "slow_1", "fast_slope", "slow_slope",
                  "t_line", "t_line_n", "t_line_1", "t_slope", 
                  "fast_length1", "slow_length_1", "fast_length", "slow_length",
                  "macd", "macd_1", "macd_slope", "macd_9" ])


# get time in minutes from data start
min1Data.EpochTime=(min1Data.index-min1Data.index[0]).total_seconds()
min5Data.EpochTime=(min5Data.index-min5Data.index[0]).total_seconds()

#Start loop from maximum period to ensure each EMA function has time to spool up
goodTime=min1Data.EpochTime[min1Data.EpochTime>maxPeriod] 
# goodTime=goodTime[:2335-590] # doing this because we are failing due to slightly less 5min data
for iter,t in enumerate(goodTime):
    if iter>len(goodTime)-6:
        break
    i=np.argmax(t==min1Data.EpochTime) # get index of time vector
    # get data to put in EMA
    tempPrice=min1Data.close.iloc[i] # price at time t
    temp1Data=min1Data.iloc[:i] # historical values at time t

    # get 5 min data
    ind5=(np.argmax(min5Data.EpochTime>t)) # find index in 5 min data which is > than t
    temp5Price=min5Data.close.iloc[(ind5-1)] # get last 5 minute data price
    temp5Data=min5Data[:(ind5)] # get historical 5 minute values at time t

    # loop through each time step for 1 and 5 minute 
    temp = IndicatorCalculations.calcEMA(temp1Data, temp5Data, tempPrice, nVec=nVec) # calculate EMA
    
    #################### Check to Enter ####################
    if not context.enter_flag: # if we dont have an order for stock
        EnterExits.checkEnter(context,temp,tempPrice) # check if we want to enter
    if context.enter_flag and not context.entry_flag: # Place buy order
        context.entry_time = context.sTime 
        if context.shortBool:
            context.short_flag = True   
        else:
            context.long_flag=True
        context.entry_flag =  True           
        context.longBool =    False
        context.shortBool =   False
        context.enter_flag =  False
        
    #################### Check to Exit ####################
    if context.entry_flag: 
        EnterExits.checkExit(context, temp, tempPrice) # check if we want to exit
    if context.exit_flag: 
        context.exit_order = None 
        context.exit_flag = False
        Utils.flag_reset(context)

            
    # append dataframe with new row of data
    ema=ema.append(temp)
    price.append(tempPrice)
    time.append(i)
    enter_flag.append(context.enter_flag)
    entry_flag.append(context.entry_flag)
    exit_flag.append(context.exit_flag)
    long_bool.append(context.longBool)
    short_bool.append(context.shortBool)
ema.index=np.arange(len(ema.index))
ema.insert(0,"time",time)
ema.insert(1,"price",price)
ema.insert(len(ema.columns),"enter_flag",enter_flag)
ema.insert(len(ema.columns),"entry_flag",entry_flag)
ema.insert(len(ema.columns),"exit_flag",exit_flag)
ema.insert(len(ema.columns),"long_bool",long_bool)
ema.insert(len(ema.columns),"short_bool",short_bool)
print("done")

In [ ]:
def calcEMA(min1Data, min5Data, price: float, nVec: list = [13, 48, 8]):
    """This function will take in your 1 minute data, 5 minute data (must both be dataFrames),and n vector 
    (vector containing periods for fast, slow, and Tline) and output a structure containing all relevant EMAs"""
    
    assert len(nVec)==3, "N vector must have 3 periods for fast, slow, and T line"
       
    # calculate all the EMAs
    
    fast = EMA(min1Data.close, price, nVec[0])
    fast_n = EMA(min1Data.close, price, nVec[0])
    fast_1 = EMA_1(min1Data.close, nVec[0])
    fast_2 = EMA_1(min1Data.close, nVec[0])

    slow = EMA(min1Data.close, price, nVec[1])
    slow_n = EMA(min1Data.close, price, nVec[1])
    slow_1 = EMA_1(min1Data.close, nVec[1])
    fast_slope = round((fast_n - fast_2)/2, 4)
    slow_slope = round((slow_n - slow_1) / 2, 4)

    t_line = EMA(min1Data.close, price, nVec[0])
    t_line_n = EMA(min1Data.close, price, nVec[2])
    t_line_1 = EMA_1(min1Data.close, nVec[2])
    t_slope = round((t_line_n-t_line_1) / 2, 4)
    
    fast_length_1 = EMA_1(min5Data.close, 12)
    slow_length_1 = EMA_1(min5Data.close, 26)
    fast_length = EMA_1(min5Data.close, 12)
    slow_length = EMA_1(min5Data.close, 26)

    macd_1 = fast_length_1 - slow_length_1
    macd = round(fast_length - slow_length, 2) # MACD at time T
    macd_slope = round((macd-macd_1)/2, 4)
    macd_9 = MACD(min5Data, macd, 9) # signal line

In [ ]:
class ExpMovAvg():
    
    def __init__(self,min1Data, min5Data, price: float, nVec: list = [13, 48, 8]):
        
        assert len(nVec)==3, "N vector must have 3 periods for fast, slow, and T line"
       
        # calculate all the EMAs

        self.fast = EMA(min1Data.close, price, nVec[0])
        self.fast_n = EMA(min1Data.close, price, nVec[0])
        self.fast_1 = EMA_1(min1Data.close, nVec[0])
        self.fast_2 = EMA_1(min1Data.close, nVec[0])

        self.slow = EMA(min1Data.close, price, nVec[1])
        self.slow_n = EMA(min1Data.close, price, nVec[1])
        self.slow_1 = EMA_1(min1Data.close, nVec[1])
        self.fast_slope = round((fast_n - fast_2)/2, 4)
        self.slow_slope = round((slow_n - slow_1) / 2, 4)

        self.t_line = EMA(min1Data.close, price, nVec[0])
        self.t_line_n = EMA(min1Data.close, price, nVec[2])
        self.t_line_1 = EMA_1(min1Data.close, nVec[2])
        self.t_slope = round((t_line_n-t_line_1) / 2, 4)

        self.fast_length_1 = EMA_1(min5Data.close, 12)
        self.slow_length_1 = EMA_1(min5Data.close, 26)
        self.fast_length = EMA_1(min5Data.close, 12)
        self.slow_length = EMA_1(min5Data.close, 26)

        self.macd_1 = fast_length_1 - slow_length_1
        self.macd = round(fast_length - slow_length, 2) # MACD at time T
        self.macd_slope = round((macd-macd_1)/2, 4)
        self.macd_9 = MACD(min5Data, macd, 9) # signal line
        
        

In [ ]:
    ema=ema.append(temp)
    price.append(tempPrice)
    time.append(i)
    enter_flag.append(context.enter_flag)
    entry_flag.append(context.entry_flag)
    exit_flag.append(context.exit_flag)
    long_bool.append(context.longBool)
    short_bool.append(context.shortBool)
ema.index=np.arange(len(ema.index))
ema.insert(0,"time",time)
ema.insert(1,"price",price)
ema.insert(len(ema.columns),"enter_flag",enter_flag)
ema.insert(len(ema.columns),"entry_flag",entry_flag)
ema.insert(len(ema.columns),"exit_flag",exit_flag)
ema.insert(len(ema.columns),"long_bool",long_bool)
ema.insert(len(ema.columns),"short_bool",short_bool)
print("done")

In [ ]:
dailyLog.write(context.sTime, ema.fast, ema.slow, ema.t_line, ema.macd, ema.macd_9, 
 context.longBool, context.shortBool, context.enter_flag, context.entry_flag,
 price.ask, price.bid, price.last, price.mid)

In [ ]:
os.chdir(r'D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy')

log_file_name = r'D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\log_testing.csv'
with open(log_file_name, 'a', newline='') as dailyLog:
    dailyLog.write("time, fast, slow, tLine, macd, signalLine, longBool, shortBool, enterFlag, entryFlag, askPrice, bidPrice, lastPrice, midPrice")


In [ ]:
time, fast, slow, tLine, macd, signalLine, longBool, shortBool, enterFlag, entryFlag, askPrice, bidPrice, lastPrice, midPrice

In [ ]:
Friday:

# write function to collect data at end of day
data.parentTrader class for testing
plotter end of day function
auto email function

Saturday: 

unit tests
algorithm tweaking

Sunday:

algorithm tweaking
autologin

In [ ]:
context = testutils.context(sim)
context.sTime = sim.get_datetime('US/Eastern') 

dailyPriceData = sim.request_historical_data(symbol('SPY'), '1 min', '23400 S')

dailyPriceData.to_csv(r"log\Daily_Price_Data_"+str(datetime.date.today())+".csv")

In [ ]:
# load price data
priceName = r"log\Daily_Price_Data_"+str(datetime.date.today())+".csv"
priceColumns = ["date", "time", "open", "high", "low", "close", "volume"]
priceData = pd.read_csv(priceName, header=1, names=priceColumns)
priceData['date'] =  np.array([datetime.datetime.fromtimestamp(f) for f in priceData.time.values])

# load ema data
logName = r"log\Daily_Log_"+str(datetime.date.today())+".csv"
logColumns = ["time", "fast", "slow", "tLine", "macd", "signalLine", "longBool", "shortBool", 
            "enterFlag", "entryFlag", "priceAsk", "priceBid", "priceLast", "priceMid"]

dictType = {"time": float,  "fast": float, "slow": float, "tLine": float, "macd": float, "signalLine": float, "longBool": bool, "shortBool": bool, 
            "enterFlag": bool, "entryFlag": bool, "priceAsk": float, "priceBid": float, "priceLast": float, "priceMid": float}

logData = pd.read_csv(logName, header=1, names=logColumns)
# logData = logData[logData.time!="time"] # remove extra rows resulting from rerunning algo
# logData.time = logData.time.astype(float)
# logData.fast = logData.time.astype(float)
# logData.slow = logData.time.astype(float)
# logData.tLine = logData.time.astype(float)
# logData.macd = logData.time.astype(float)
# logData.signalLine = logData.time.astype(float)
logData['date'] = np.array([datetime.datetime.fromtimestamp(float(f)) for f in logData.time.values])
# plot the candlestick
# plot the ema

# add the MACD

In [ ]:
#create figure
emaList = ['fast', 'slow']
fig, ax = plt.subplots(figsize=(18,14))

#define width of candlestick elements
width = 0.8
width2 = 0.05

#define up and down priceData
up = priceData[priceData.close>=priceData.open]
down = priceData[priceData.close<priceData.open]

#define colors to use
col1 = 'green'
col2 = 'red'

for i,j in enumerate(emaList):
    exec('temp{}=logData.{}'.format(i,j))    
    exec('ax.plot(logData.time, temp{})'.format(i))


# #plot up priceData
ax.bar(up.time,up.close-up.open,width,bottom=up.open,color=col1)
ax.bar(up.time,up.high-up.close,width2,bottom=up.close,color=col1)
ax.bar(up.time,up.low-up.open,width2,bottom=up.open,color=col1)

#plot down priceData
ax.bar(down.time,down.close-down.open,width,bottom=down.open,color=col2)
ax.bar(down.time,down.high-down.open,width2,bottom=down.open,color=col2)
ax.bar(down.time,down.low-down.close,width2,bottom=down.close,color=col2)

ax.legend(emaList)
ax.grid(which='major')
ax.grid(which='minor', linewidth=0.25)
ax.minorticks_on()
# ax.set(xlabel="Minutes After Opening", ylabel="Stock Price (USD)", 
#    title=security+" Stock Price: "+date)

# ax2=ax.twinx()
# ax2.plot(ema.time,ema.macd)

fig.savefig(r'D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Output\test.png')

In [ ]:
### plotting with dates is a pain in the ass, come back to this later

from dateutil import tz
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
emaList = ['fast', 'slow']
#create figure
fig, ax = plt.subplots(figsize=(36,28))

#define width of candlestick elements
width = 0.0005
width2 = 0.0001

#define up and down priceData
up = priceData[priceData.close>=priceData.open]
down = priceData[priceData.close<priceData.open]

#define colors to use
col1 = 'green'
col2 = 'red'

for i,j in enumerate(emaList):
    exec('temp{}=logData.{}'.format(i,j))    
    exec('ax.plot(logData.date, temp{})'.format(i))


#plot up priceData
ax.bar(up.date,up.close-up.open,width,bottom=up.open,color=col1)
ax.bar(up.date,up.high-up.close,width2,bottom=up.close,color=col1)
ax.bar(up.date,up.low-up.open,width2,bottom=up.open,color=col1)

# #plot down priceData
ax.bar(down.date,down.close-down.open,width,bottom=down.open,color=col2)
ax.bar(down.date,down.high-down.open,width2,bottom=down.open,color=col2)
ax.bar(down.date,down.low-down.close,width2,bottom=down.close,color=col2)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M',tz=tz.gettz('America/New York')))


# Turn grid on for both major and minor ticks and style minor slightly differently.
ax.grid(which='major')
ax.grid(which='minor', linewidth=0.25)
ax.minorticks_on()

ax.set(xlabel="Time", ylabel="Stock Price (USD)", 
   title="SPY Stock Price: "+str(datetime.date.today()),
       xlim = (datetime.datetime(2022,11,11,7,31,0),datetime.datetime(2022,11,11,9,31,0)))


fig.savefig(r'D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Output\test.png')

In [ ]:

plt.bar(up.date,up.close, width = .001)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M',tz=tz.gettz('America/New York')))
# ax.show

In [ ]:
logData = pd.read_csv(logName, header=1, names=logColumns)
# logData = logData[logData.time!="time"] # remove extra rows resulting from rerunning algo
# logData.time = logData.time.astype(float)
# logData.fast = logData.time.astype(float)
# logData.slow = logData.time.astype(float)
# logData.tLine = logData.time.astype(float)
# logData.macd = logData.time.astype(float)
# logData.signalLine = logData.time.astype(float)


In [ ]:
# datetime.datetime.fromtimestamp(1668177060)
datetime.datetime(2022,11,11,7,31,0)

In [ ]:
def plotDailyCandleStickMacd(inputData, 
                 ema, 
                 emaList: list = ['fast', 'slow', "macd"], 
                 security: str = "SPY",
                 date: str = "01/15/2021",
                 saveFig: bool = False,
                 outPutFileName: str=r"C:\Users\Lane Capital\OneDrive\Desktop\repos\plots\TestCandle.png"):
    
    """This function takes in a historical input data dataframe, EMA dataframe from calcEMAVector 
    and plots the differnet EMAS against the price. Price is assumed to be the close price"""

    # TODO figure out how to get kwargs working for date and outputfilename
    # TODO get ylim as a function of data plotted
    # TODO get xlim as an input

    #create figure
    fig, ax = plt.subplots(figsize=(18,14))

    #define width of candlestick elements
    width = 0.5
    width2 = 0.05

    #define up and down inputData
    up = inputData[inputData.close>=inputData.open]
    down = inputData[inputData.close<inputData.open]

    #define colors to use
    col1 = 'green'
    col2 = 'red'

    for i,j in enumerate(emaList):
        exec('temp{}=ema.{}'.format(i,j))    
        exec('ax.plot(ema.time, temp{})'.format(i))


    #plot up inputData
    ax.bar(up.index,up.close-up.open,width,bottom=up.open,color=col1)
    ax.bar(up.index,up.high-up.close,width2,bottom=up.close,color=col1)
    ax.bar(up.index,up.low-up.open,width2,bottom=up.open,color=col1)

    #plot down inputData
    ax.bar(down.index,down.close-down.open,width,bottom=down.open,color=col2)
    ax.bar(down.index,down.high-down.open,width2,bottom=down.open,color=col2)
    ax.bar(down.index,down.low-down.close,width2,bottom=down.close,color=col2)

    ax.legend(emaList)
    ax.grid(which='major')
    ax.grid(which='minor', linewidth=0.25)
    ax.minorticks_on()
    ax.set(xlabel="Minutes After Opening", ylabel="Stock Price (USD)", 
       title=security+" Stock Price: "+date)
    
    ax2=ax.twinx()
    ax2.plot(ema.time,ema.macd)

    if saveFig:
        fig.savefig(outPutFileName)

In [ ]:
def plotDailyCandleStick(priceData, 
                 logData, 
                 emaList: list = ['fast', 'slow', "macd"],
                 saveFig: bool = False,
                 outPutFileName: str=r"C:\Users\Lane Capital\OneDrive\Desktop\repos\plots\TestCandle.png",
                 x = None):
    
    """This function takes in a historical input data dataframe, EMA dataframe from calcEMAVector 
    and plots the differnet EMAS against the price. Price is assumed to be the close price"""

    #create figure
    fig, ax = plt.subplots(figsize=(36,28))

    #define width of candlestick elements
    width = 0.0005
    width2 = 0.0001

    #define up and down priceData
    up = priceData[priceData.close>=priceData.open]
    down = priceData[priceData.close<priceData.open]
    log = logData.copy()

    # remove elements not in the x limit
    up = up[up.time>datetime.datetime.timestamp(x[0])]
    up = up[up.time<datetime.datetime.timestamp(x[1])]

    down = down[down.time>datetime.datetime.timestamp(x[0])]
    down = down[down.time<datetime.datetime.timestamp(x[1])]

    log = log[log.time>datetime.datetime.timestamp(x[0])]
    log = log[log.time<datetime.datetime.timestamp(x[1])]

    #define colors to use
    col1 = 'green'
    col2 = 'red'

    for i,j in enumerate(emaList):
        exec('temp{}=log.{}'.format(i,j))    
        exec('ax.plot(log.date, temp{})'.format(i))

    outArray=plotEntryStatus(log)
    ax2 = ax.twinx()
    ax2.plot(log.date,outArray,color='g')
    fig.legend(['fast','slow','holding flag'],fontsize=30) # sticking this code here as a lazy fix for the legend

    #plot up priceData
    ax.bar(up.date,up.close-up.open,width,bottom=up.open,color=col1)
    ax.bar(up.date,up.high-up.close,width2,bottom=up.close,color=col1)
    ax.bar(up.date,up.low-up.open,width2,bottom=up.open,color=col1)

    # #plot down priceData
    ax.bar(down.date,down.close-down.open,width,bottom=down.open,color=col2)
    ax.bar(down.date,down.high-down.open,width2,bottom=down.open,color=col2)
    ax.bar(down.date,down.low-down.close,width2,bottom=down.close,color=col2)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M',tz=tz.gettz('America/New York')))


    # Turn grid on for both major and minor ticks and style minor slightly differently.
    ax.grid(which='major')
    ax.grid(which='minor', linewidth=0.25)
    ax.minorticks_on()
    plt.ylim(np.min([np.min(up.low),np.min(down.low)])-.05,np.max([np.max(up.high),np.max(down.high)])+.05)
    if x==None:
        plt.xlabel("Time",fontsize=30)
        plt.ylabel("Stock Price (USD)",fontsize=30)
        plt.title("SPY Stock Price: "+str(datetime.date.today()),fontdict={'fontsize': 40})
    else:
        ax.set(xlim = x)
        plt.xlabel("Time",fontsize=30)
        plt.ylabel("Stock Price (USD)",fontsize=30)
        plt.title("SPY Stock Price: "+str(datetime.date.today()),fontdict={'fontsize': 40})
           
    outArray=plotEntryStatus(log)
    ax2 = ax.twinx()
    ax2.plot(log.date,outArray,color='g')
    if saveFig:
        fig.savefig(outPutFileName)
        
# plotDailyCandleStick(priceData, logData, emaList = ['fast', 'slow'])

In [ ]:
xlim = [(datetime.datetime(2022,11,11,7,31,0),datetime.datetime(2022,11,11,8,31,0)),
        (datetime.datetime(2022,11,11,8,31,0),datetime.datetime(2022,11,11,9,31,0)),
        (datetime.datetime(2022,11,11,9,31,0),datetime.datetime(2022,11,11,10,31,0)),
        (datetime.datetime(2022,11,11,10,31,0),datetime.datetime(2022,11,11,11,31,0)),
        (datetime.datetime(2022,11,11,11,31,0),datetime.datetime(2022,11,11,12,31,0)),
        (datetime.datetime(2022,11,11,12,31,0),datetime.datetime(2022,11,11,13,31,0)),
        (datetime.datetime(2022,11,11,13,31,0),datetime.datetime(2022,11,11,14,31,0)),
        (datetime.datetime(2022,11,11,14,31,0),datetime.datetime(2022,11,11,15,31,0)),
        (datetime.datetime(2022,11,11,15,31,0),datetime.datetime(2022,11,11,16,1,0))]

for x in xlim:
    outputFileName = r"D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Output\candlestick_"+str(datetime.date.today())+"_"+str(x[0].hour)+".png"
    Plotters.plotDailyCandleStick(priceData, logData, emaList = ['fast', 'slow'], x=x,outPutFileName=outputFileName,saveFig=True)

In [ ]:
xlim = [(datetime.datetime(2022,11,11,7,31,0),datetime.datetime(2022,11,11,8,31,0)),
        (datetime.datetime(2022,11,11,8,31,0),datetime.datetime(2022,11,11,9,31,0)),
        (datetime.datetime(2022,11,11,9,31,0),datetime.datetime(2022,11,11,10,31,0)),
        (datetime.datetime(2022,11,11,10,31,0),datetime.datetime(2022,11,11,11,31,0)),
        (datetime.datetime(2022,11,11,11,31,0),datetime.datetime(2022,11,11,12,31,0)),
        (datetime.datetime(2022,11,11,12,31,0),datetime.datetime(2022,11,11,13,31,0)),
        (datetime.datetime(2022,11,11,13,31,0),datetime.datetime(2022,11,11,14,31,0)),
        (datetime.datetime(2022,11,11,14,31,0),datetime.datetime(2022,11,11,15,31,0)),
        (datetime.datetime(2022,11,11,15,31,0),datetime.datetime(2022,11,11,16,1,0))]
x = xlim[4]

emaList: list = ['fast', 'slow']



#create figure
fig, ax = plt.subplots(figsize=(36,28))

#define width of candlestick elements
width = 0.0005
width2 = 0.0001

#define up and down priceData
up = priceData[priceData.close>=priceData.open]
down = priceData[priceData.close<priceData.open]
log = logData.copy()

# remove elements not in the x limit
up = up[up.time>datetime.datetime.timestamp(x[0])]
up = up[up.time<datetime.datetime.timestamp(x[1])]

down = down[down.time>datetime.datetime.timestamp(x[0])]
down = down[down.time<datetime.datetime.timestamp(x[1])]

log = log[log.time>datetime.datetime.timestamp(x[0])]
log = log[log.time<datetime.datetime.timestamp(x[1])]

#define colors to use
col1 = 'green'
col2 = 'red'

for i,j in enumerate(emaList):
    exec('temp{}=log.{}'.format(i,j))    
    exec('ax.plot(log.date, temp{})'.format(i))
    
outArray=plotEntryStatus(log)
ax2 = ax.twinx()
ax2.plot(log.date,outArray,color='g')
fig.legend(['fast','slow','holding flag'],fontsize=30) # sticking this code here as a lazy fix for the legend

#plot up priceData
ax.bar(up.date,up.close-up.open,width,bottom=up.open,color=col1)
ax.bar(up.date,up.high-up.close,width2,bottom=up.close,color=col1)
ax.bar(up.date,up.low-up.open,width2,bottom=up.open,color=col1)

# #plot down priceData
ax.bar(down.date,down.close-down.open,width,bottom=down.open,color=col2)
ax.bar(down.date,down.high-down.open,width2,bottom=down.open,color=col2)
ax.bar(down.date,down.low-down.close,width2,bottom=down.close,color=col2)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M',tz=tz.gettz('America/New York')))


# Turn grid on for both major and minor ticks and style minor slightly differently.
ax.grid(which='major')
ax.grid(which='minor', linewidth=0.25)
ax.minorticks_on()
plt.ylim(np.min([np.min(up.low),np.min(down.low)])-.05,np.max([np.max(up.high),np.max(down.high)])+.05)
if xlim==None:
    plt.xlabel("Time",fontsize=30)
    plt.ylabel("Stock Price (USD)",fontsize=30)
    plt.title("SPY Stock Price: "+str(datetime.date.today()),fontdict={'fontsize': 40})
else:
    ax.set(xlim = x)
    plt.xlabel("Time",fontsize=30)
    plt.ylabel("Stock Price (USD)",fontsize=30)
    plt.title("SPY Stock Price: "+str(datetime.date.today()),fontdict={'fontsize': 40})
ax2 = ax.twinx()
ax2.plot(log.date,outArray,color='g')



In [ ]:
fig, ax = plt.subplots()
ax.plot(log.date,log.longBool)
ax.plot(log.date,log.shortBool)
ax.plot(log.date,log.enterFlag)
ax.plot(log.date,log.entryFlag)
plt.savefig(r"D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Output\test.png")

In [ ]:
plt.plot(testArray)

In [ ]:
log.longBool = log.longBool.astype(bool)
log.shortBool = log.shortBool.astype(bool)



In [ ]:
def fixBooldata(logData):
    """ This function takes in the daily log data and converts the long/shortbool
    data and converts it to boolean from string format."""
    
    # clean up short data
    for i in range(len(logData.shortBool.values)):
        if logData.shortBool.iloc[i].split()[0].lower()=="false":
            logData.shortBool.iloc[i]=False
        elif logData.shortBool.iloc[i].split()[0].lower()=="true":
            logData.shortBool.iloc[i]=True
        else:
            print("Error, string is not true or false")
    
    # clean up long data
    for i in range(len(logData.longBool.values)):
        if logData.longBool.iloc[i].split()[0].lower()=="false":
            logData.longBool.iloc[i]=False
        elif logData.longBool.iloc[i].split()[0].lower()=="true":
            logData.longBool.iloc[i]=True
        else:
            print("Error, string is not true or false")
    return logData

def calcEntryStatus(logData):
    """ This function takes in a dataframe loaded from the daily log data
    and checks for true/false and ouputs and array with the following indices:
        1 for holding long position
        0 for holding no position
        -1 for holding short position"""
    # intialize array
    testArray = np.zeros_like(logData.longBool.values)-9999

    for i in range(len(logData.longBool.values)):
        if logData.longBool.iloc[i]==True and logData.shortBool.iloc[i]==False:
            testArray[i] = 1
        elif logData.longBool.iloc[i]==False and logData.shortBool.iloc[i]==True:
            testArray[i] = -1
        elif logData.longBool.iloc[i]==False and logData.shortBool.iloc[i]==False:
            testArray[i] = 0
            
    return testArray

def plotEntryStatus(logData):
    """ This function takes in data from the daily log, cleans the short/long bool data, and 
    outputs an array displaying if were holding long, short, or nothing."""
    
    # copy data
    log=logData.copy()
    
    # clean up data
    log = Utils.fixBooldata(log)
    
    holdStatus = Utils.calcEntryStatus(log)
    
    return holdStatus

In [ ]:
outArray=plotEntryStatus(logData)

In [ ]:
log.shortBool.iloc[0]

In [ ]:
plt.plot(outArray)